In [ ]:
def VoiceIDnet(x_train,y_train,x_test,y_test,X0):

    !pip install -q keras
    import keras
    from keras.models import Sequential
    from keras.layers import Dense, Activation, BatchNormalization
    from keras.optimizers import SGD
    from keras import regularizers, optimizers
    import numpy as np
    import tensorflow as tf

    classes = y_train.shape[1]  # num speakers
    n_vec =   x_train.shape[1]  # length of feature vector
    m_train = x_train.shape[0]  # num training examples
    m_test =  x_test.shape[0]   # num test examples
    params = {}                 # initalize params

    print ("number of training examples = " + str(m_train))
    print ("number of test examples = " + str(m_test))
    print("number of classes: " + str(classes))

    Lparam     = int(X0[0])
    bsizeparam = int(X0[1])
    L2param    = int(X0[2]*1000)/1000
    Nparam     = int(X0[3])
  
    #################### Hyperparameters ####################
    L = Lparam                       # num layers in network
    bsize = bsizeparam               # minibatch size
    epochnum = 20                    # num epochs for optimization
    learning_rate = 0.001            # params for Adam
    learning_decay = 0.00001         # learning decay for Adam
    BN_mom = 0.99                    # momentum for batch norm
    L2 = np.ones(L)*L2param          # L2 lambda parameters for each layer
    N = np.append(np.ones(L-1)*Nparam,classes)   # num nodes/layer for each hidden layer
    #########################################################
    

    for l in range(1,L+1):
        params["L2_"+str(l)] = L2[l-1]
        params["N"+str(l)] = N[l-1]

    ## NN Model (X->RELU->RELU->RELU->RELU->SOFTMAX)
    model = Sequential()
    model.add(Dense(params['N1'], activation='relu', input_dim=n_vec,use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=regularizers.l2(params["L2_1"]), bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None,name='fc1'))
    model.add(BatchNormalization(axis=1, momentum=BN_mom, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones',name='bn1'))
    for l in range(2,L-1):
        model.add(Dense(params['N'+str(l)], activation='relu',use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=regularizers.l2(params["L2_" + str(l)]), bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None,name='fc'+str(l)))
        model.add(BatchNormalization(axis=1, momentum=BN_mom, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones',name='bn'+str(l)))
    model.add(Dense(params['N'+str(L)], activation='softmax',use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=regularizers.l2(params["L2_" + str(L)]), bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None,name='fc'+str(L)))

    ## Compile and Train Model
    adam_opt = optimizers.Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=None, decay=learning_decay, amsgrad=False)
    model.compile(loss='categorical_crossentropy',optimizer=adam_opt,metrics=['accuracy'])
    model.fit(x_train, y_train,epochs=epochnum,batch_size=bsize)

    ## Test Model
    preds = model.evaluate(x_test, y_test, batch_size=bsize)
    test_accuracy = preds[1]
    test_loss = preds[0]
    print ("Loss = " + str(test_loss))
    print ("Test Accuracy = " + str(test_accuracy))

    ## Model Predictions of Test Set
    y_hat = model.predict(x_test, batch_size=bsize)

    ## Display Model Architecture
    model.summary()
    
    return test_accuracy